# Steady-State Thermal Model of META

Here's the setup:

![](images/meta-steady-state-analysis-config.png)

There's two nodes, the body (i.e. primary chassis) and the solar panels. They are assumed to be thermally isolated. The equations boil down to:

$$
Q_\text{net} = 0 = Q_\text{sun} + Q_\text{ground} + Q_\text{sky}
$$,

where

$$
\begin{align*}
Q_\text{sun} &= \alpha A G_\text{SC} \\
Q_i &= \epsilon_i A_i \sigma (T_i^4 - T^4)
\end{align*}
$$

where $G_\text{SC}$ is the incidence solar flux at Mercury (assumed to be worst case at perihelion).

In [1]:
from scipy.optimize import root_scalar


def DEGC_TO_K(degC: float) -> float:
    return degC + 273.15


def K_TO_DEGC(K: float) -> float:
    return K - 273.15


SOLAR_FLUX = 14462  # [W/m^2], Maximum at perihelion
STEF_BOLTZ = 5.6703e-8  # [W/(m^2 K^4)]

# View Factors
F_GROUND_TO_BOTTOM = 1
F_GROUND_TO_VERT = 0.5
F_SKY_TO_TOP = 1
F_SKY_TO_VERT = 0.5


In [2]:
# Temperatures
T_GROUND = DEGC_TO_K(50)
T_SKY = 3

# Material Properties
# Body
# fmt: off
ABSORPTIVITY_BODY_FRONT     = 0.3       # Conservative value for white paint on chassis front
EMISSIVITY_BODY_FRONT       = 0.88      # Estimate for chassis front
EMISSIVITY_BODY_BOTTOM      = 0.95      # Radiator on bottom
EMISSIVITY_BODY_TOP         = 0.95      # Radiator on top
EMISSIVITY_BODY_BACKSIDE    = 0.9       # Radiator on sides and back
# Panels
ABSORPTIVITY_PANEL_FRONT    = 0.8       # Picked from range of values from NASA Publication 1121
EMISSIVITY_PANEL_FRONT      = 0.8       # Same ^
EMISSIVITY_PANEL_BACK       = 0.95      # Radiator on back of solar panels
EFFICIENCY_PANEL            = 0.2       # Estimate of how much absorbed energy is used for power
# fmt: on

# Body Dimensions
X_DIM = 0.3  # [m]
Y_DIM = 0.2  # [m]
Z_DIM = 0.2  # [m]

# Body Areas
A_BODY_FRONT = Y_DIM * Z_DIM
A_BODY_BOTTOM = X_DIM * Y_DIM
A_BODY_TOP = A_BODY_BOTTOM
A_BODY_BACKSIDE = 2 * (X_DIM * Z_DIM) + (Y_DIM * Z_DIM)


def Q_body(T_body: float) -> float:
    """Calculate net power leaving body / chassis."""
    T_body = DEGC_TO_K(T_body)

    # fmt: off
    C_GROUND = STEF_BOLTZ * (
        A_BODY_BOTTOM   * F_GROUND_TO_BOTTOM    * EMISSIVITY_BODY_BOTTOM    +
        A_BODY_BACKSIDE * F_GROUND_TO_VERT      * EMISSIVITY_BODY_BACKSIDE  +
        A_BODY_FRONT    * F_GROUND_TO_VERT      * EMISSIVITY_BODY_FRONT
    )
    C_SKY = STEF_BOLTZ * (
        A_BODY_TOP      * F_SKY_TO_TOP  * EMISSIVITY_BODY_TOP       +
        A_BODY_BACKSIDE * F_SKY_TO_VERT * EMISSIVITY_BODY_BACKSIDE  +
        A_BODY_FRONT    * F_SKY_TO_VERT * EMISSIVITY_BODY_FRONT
    )
    # fmt: on

    Q_sun = ABSORPTIVITY_BODY_FRONT * A_BODY_FRONT * SOLAR_FLUX
    Q_ground = C_GROUND * (T_GROUND**4 - T_body**4)
    Q_sky = C_SKY * (T_SKY**4 - T_body**4)
    return Q_sun + Q_ground + Q_sky


def Q_panels(T_panel: float, A_PANEL: float) -> float:
    """Calculate net power leaving solar panel assembly."""
    T_panel = DEGC_TO_K(T_panel)

    # fmt: off
    C_GROUND = STEF_BOLTZ * A_PANEL * F_GROUND_TO_VERT * (
        EMISSIVITY_PANEL_FRONT + EMISSIVITY_PANEL_BACK
    )
    C_SKY = STEF_BOLTZ * A_PANEL * F_SKY_TO_VERT * (
        EMISSIVITY_PANEL_FRONT + EMISSIVITY_PANEL_BACK
    )
    # fmt: on

    Q_sun = ABSORPTIVITY_PANEL_FRONT * (1 - EFFICIENCY_PANEL) * A_PANEL * SOLAR_FLUX
    Q_ground = C_GROUND * (T_GROUND**4 - T_panel**4)
    Q_sky = C_SKY * (T_SKY**4 - T_panel**4)
    return Q_sun + Q_ground + Q_sky


In [3]:
# Panel Area
A_PANEL = 2 * (0.3 * Z_DIM)

T_body = root_scalar(Q_body, bracket=(-300, 1000)).root
T_panel = root_scalar(Q_panels, args=(A_PANEL,), bracket=(-300, 1000)).root

print(f"T_body  = {T_body:>7.2f} degC")
print(f"T_panel = {T_panel:>7.2f} degC")

T_body  =   81.90 degC
T_panel =  287.39 degC
